In [ ]:
import os
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from math import ceil
from multiprocessing import Pool

In [ ]:
raw_dir = "plants/raw/"
file_list = os.listdir(raw_dir)
class_name = []
for name in file_list:
    if not name.startswith('.'):
        class_name.append(name)

In [ ]:
augmented_dir = "plants/augmented_multi/"

if not os.path.exists(augmented_dir):
        os.mkdir(augmented_dir)
        
for name in class_name:
    class_dir = augmented_dir + name + "/"
    if not os.path.exists(class_dir):
        os.mkdir(class_dir)

In [ ]:
target = 3000

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        horizontal_flip=True,
        fill_mode='reflect')

def augmentation(name):
    print("Start Augmenting " + name)
    class_dir = raw_dir + name +"/"
    augmented_class_dir = augmented_dir + name + "/"
    
    file_list = os.listdir(class_dir)
    augmented_list = os.listdir(augmented_class_dir)
    
    for file_name in file_list:
        if not file_name.endswith(".png"):
            file_list.remove(file_name)
            print("Removed_1")
    
    num_augment = ceil(target / len(file_list)) + 1
    
    for file_name in file_list:
        if not file_name.endswith(".png"):
            continue

        file_dir = class_dir + file_name
        image = cv2.imread(file_dir)
        print("1")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = image[...,::-1] # BGR2RGB
        print(image.shape)
        image = cv2.resize(image,(100,100))
        print("2")
        height, width, channel = image.shape
        
        if channel != 3:
            print("Removed_2")
            continue
            
        x = np.array(image).reshape([1,height,width,channel])
    
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir=augmented_class_dir, save_prefix='aug_', save_format='png'):
            i += 1
            if i >= num_augment:
                break  # otherwise the generator would loop indefinitely

    print("Finished Augmenting " + name)

In [ ]:
p = Pool()
p.map(augmentation, [name for name in class_name])